In [1]:
import dspy
from dspy.teleprompt import MIPROv2
import utils
import os
import typing
import json

from BasicHtmlToTextParser import BasicHtmlToTextParser
from metrics import stixnet_f1, semantic_match_hungarian
from sentence_transformers import SentenceTransformer
from collections import defaultdict


from pydantic import BaseModel, Field
from pydantic_core import ValidationError


from dotenv import load_dotenv
_ = load_dotenv("../.env")

In [2]:
dataset = utils.get_dspy_examples_randomized("../LADDER-Dataset/", BasicHtmlToTextParser(include_images=False), random_seed=1337)

trainset, devset = utils.split_dataset(split_at=47, dataset=dataset)

trainset, devset = utils.generate_targets_dataset(trainset), utils.generate_targets_dataset(devset)

trainset_ids, devset_ids = [example.id for example in trainset], [example.id for example in devset]

47 100 total: 147 train-ratio: 0.3197278911564626 dev-ratio: 0.6802721088435374


In [3]:
llama_3_1_8b_instruct_deepinfra = dspy.LM(model="openai/meta-llama/Meta-Llama-3.1-8B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.1, max_tokens=1024)

llama_3_1_8b_instruct_no_cache_deepinfra = dspy.LM(model="openai/meta-llama/Meta-Llama-3.1-8B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.3, max_tokens=1024, cache=False, cache_in_memory=False)




llama_3_1_70b_instruct_turbo_deepinfra = dspy.LM(model="openai/meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.1, max_tokens=1024)

llama_3_1_70b_instruct_turbo_no_cache_deepinfra = dspy.LM(model="openai/meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.3, max_tokens=1024, cache=False, cache_in_memory=False)



llama_3_2_3b_instruct_deepinfra = dspy.LM(model="openai/meta-llama/Llama-3.2-3B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.1, max_tokens=1024)

llama_3_2_3b_instruct_no_cache_deepinfra = dspy.LM(model="openai/meta-llama/Llama-3.2-3B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.3, max_tokens=1024, cache=False, cache_in_memory=False)



llama_3_2_1b_instruct_deepinfra = dspy.LM(model="openai/meta-llama/Llama-3.2-1B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.1, max_tokens=1024)

llama_3_2_1b_instruct_no_cache_deepinfra = dspy.LM(model="openai/meta-llama/Llama-3.2-1B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.3, max_tokens=1024, cache=False, cache_in_memory=False)






# qwen 
qwen_2_5_7b_instruct_deepinfra = dspy.LM(model="openai/Qwen/Qwen2.5-7B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.1, max_tokens=1024)

qwen_2_5_7b_instruct_no_cache_deepinfra = dspy.LM(model="openai/Qwen/Qwen2.5-7B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.3, max_tokens=1024, cache=False, cache_in_memory=False)


qwen_2_5_72b_instruct_deepinfra = dspy.LM(model="openai/Qwen/Qwen2.5-72B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.1, max_tokens=1024)

qwen_2_5_72b_instruct_no_cache_deepinfra = dspy.LM(model="openai/Qwen/Qwen2.5-72B-Instruct", api_key=os.environ.get("DEEPINFRA_API_KEY"), base_url="https://api.deepinfra.com/v1/openai", temperature=0.3, max_tokens=1024, cache=False, cache_in_memory=False)

In [4]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", token=os.environ.get("HF_API_KEY"))

In [5]:
from sentence_transformers import util

sentences = ["Adobe Flash-Player", "Adobe Flash-Player Version 2.4"]

embedding_1 = model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2)

tensor([[0.9383]])

In [6]:
PRECISION_STORE, RECALL_STORE, F1_STORE = [], [], []

def metric(example, pred, trace=None): 
    
    malwares_with_target = set()
    for triple in example.targets_triples: 
        malwares_with_target.add(triple.source)

    gold_malwares = [malware.lower() for malware in malwares_with_target]
    pred_malwares = list({triple.source.lower() for triple in pred.targets_triples})
    malware_matches = list(set(gold_malwares) & set(pred_malwares))



    gold_target_locations_by_malware = defaultdict(set)
    gold_target_software_by_malware = defaultdict(set)
    for triple in example.targets_triples:
        source_malware = triple.source.lower()
        assert triple.source_type == "malware"

        if triple.target_type == "location":
            gold_target_locations_by_malware[source_malware].add(triple.target)
        elif triple.target_type == "tool":
            gold_target_software_by_malware[source_malware].add(triple.target)
       

    pred_target_locations_by_malware = defaultdict(set)
    pred_target_software_by_malware = defaultdict(set)
    for triple in pred.targets_triples:
        source_malware = triple.source.lower()
        assert triple.source_type == "malware"

        if triple.target_type == "location":
            pred_target_locations_by_malware[source_malware].add(triple.target)
        elif triple.target_type == "tool":
            pred_target_software_by_malware[source_malware].add(triple.target)
    
    total_tp, total_fp, total_fn = 0, 0, 0 
    
    for malware in malware_matches: 
        gold_locations = gold_target_locations_by_malware.get(malware, set())
        pred_locations = pred_target_locations_by_malware.get(malware, set())

        tp_loc, fp_loc, fn_loc = semantic_match_hungarian(pred_locations, gold_locations, model=model, threshold=0.9)

        gold_software = gold_target_software_by_malware.get(malware, set())
        pred_software = pred_target_software_by_malware.get(malware, set())

        tp_soft, fp_soft, fn_soft = semantic_match_hungarian(pred_software, gold_software, model=model, threshold=0.9)

        total_tp += tp_loc + tp_soft
        total_fp += fp_loc + fp_soft
        total_fn += fn_loc + fn_soft


    missing_malwares = set(gold_malwares) - set(pred_malwares)
    for missing_malware in missing_malwares:
        total_fn += len(gold_target_locations_by_malware.get(missing_malware, []))
        total_fn += len(gold_target_software_by_malware.get(missing_malware, []))

    extra_malwares = set(pred_malwares) - set(gold_malwares)
    for extra_malware in extra_malwares:
        total_fp += len(pred_target_locations_by_malware.get(extra_malware, []))
        total_fp += len(pred_target_software_by_malware.get(extra_malware, []))


    precision, recall, f1 = stixnet_f1(total_tp, total_fp, total_fn)

    PRECISION_STORE.append(precision)
    RECALL_STORE.append(recall)
    F1_STORE.append(f1)
    return f1


def save_and_evaluate(program: dspy.Predict, llm: dspy.LM, llm_no_cache: dspy.LM, llm_id: str, base_path: str, valset: typing.List[dspy.Example]):
    # #################################################################################################
    global PRECISION_STORE
    global RECALL_STORE
    global F1_STORE

    PRECISION_STORE, RECALL_STORE, F1_STORE = [], [], []
    # #################################################################################################
    
    retry_stats = []
    for i, obj in enumerate(valset): 
        
        print(f"{i+1}/{len(valset)}")

        retries, max_retries = 0, 5
        while True:
            try: 
                print(f"Retry {retries}/{max_retries}")
                if retries > 0:
                    with dspy.settings.context(lm=llm_no_cache):
                        targets_triples = program(**obj.inputs()).targets_triples
                else:
                    with dspy.settings.context(lm=llm):
                        targets_triples = program(**obj.inputs()).targets_triples

                
                pred = dspy.Prediction(targets_triples=targets_triples)

                f1 = metric(obj, pred)
                retry_stats.append({"finished": True, "retries": retries})
                print(f"✔️ done with {retries} retries")
                break

            except ValidationError as e:
                retries += 1
                if retries == max_retries:  
                    retry_stats.append({"finished": False, "retries": retries})
                    RECALL_STORE.append(0)
                    PRECISION_STORE.append(0)
                    F1_STORE.append(0)
                    print(f"❌ Failed after {retries} retries")
                    break
                
            except Exception as e: 
                retries += 1
                if retries == max_retries:  
                    retry_stats.append({"finished": False, "retries": retries})
                    RECALL_STORE.append(0)
                    PRECISION_STORE.append(0)
                    F1_STORE.append(0)
                    print(f"❌ Failed after {retries} retries")
                    break    
            
    # store result
    with open(f"{base_path}/{llm_id}_precision.json", "w") as fp:
        json.dump(PRECISION_STORE, fp)

    with open(f"{base_path}/{llm_id}_recall.json", "w") as fp:
        json.dump(RECALL_STORE, fp)

    with open(f"{base_path}/{llm_id}_f1.json", "w") as fp:
        json.dump(F1_STORE, fp)

    with open(f"{base_path}/{llm_id}_retry_stats.json", "w") as fp:
        json.dump(retry_stats, fp)

    return retry_stats

In [7]:
for example in devset + trainset:
    assert metric(example, example) == 1.0

In [8]:
class triple(BaseModel):
    source: str = Field(description="name of the STIX Domain Object (SDO)")
    source_type: typing.Literal["attack_pattern", "campaign", "intrusion_set", "malware", "threat_actor"] = Field(description="type of the SDO")

    relationship: typing.Literal["targets"] = Field(description="STIX Relationship Object (SRO) that connects the source object with the target")

    target: str = Field(description="name of the STIX Domain Object (SDO) that is the target. Written exactly as in the threat_report (like a directly cited quote)")
    target_type: typing.Literal["identity", "location", "vulnerability", "infrastructure", "tool"] = Field(description="type of the SDO")



def enforce_stix(attack_pattern_triples: typing.List[triple]):
    filered_triples = []

    for triple in attack_pattern_triples:
        if triple.source_type == "malware":
            if (triple.relationship == "targets") and (triple.target_type == "tool" or triple.target_type == "location"):
                filered_triples.append(triple)
    
    return filered_triples


class TargetsExtractionSignature(dspy.Signature):
    """
    You are a cyber threat intelligence expert and your task is to analyse the threat report and extract all meaningful targets being targeted by entities mentioned in the threat report. 

    The following triples are considered as valid as they conform to the STIX 2.1 Specification. Only extract triples that match these rules: 
        <attack_pattern, targets, identity>
        <attack_pattern, targets, location>
        <attack_pattern, targets, vulnerability>
        <attack_pattern, targets, tool>

        <campaign, targets, identity>
        <campaign, targets, location>
        <campaign, targets, vulnerability>

        <intrusion_set, targets, vulnerability>
        <intrusion_set, targets, identity>
        <intrusion_set, targets, location>

        <malware, targets, identity>
        <malware, targets, infrastructure>
        <malware, targets, vulnerability>
        <malware, targets, location>

        <threat_actor, targets, identity>
        <threat_actor, targets, location>
        <threat_actor, targets, vulnerability>

    *Final hints*
        - Make sure to stricly cite the target word by word
        - Consider mentioned_malwares, mentioned_threat_actors as hints for the source of the triples
    """
    threat_report: str = dspy.InputField()
    mentioned_malwares: typing.List[str] = dspy.InputField(desc="A list of malware names that are mentioned in the threat report")
    mentioned_threat_actors: typing.List[str] = dspy.InputField(desc="A list of threat actors that are mentioned in the threat report")
    targets_triples: typing.List[triple] = dspy.OutputField()

In [ ]:
assert False, "ALL DONE"

BASE_PATH = "./TargetsExtractor/Baseline"
mlflow.autolog()
mlflow.set_experiment(BASE_PATH)

to_do = [
    ("llama_3_2_1b_instruct", llama_3_2_1b_instruct_deepinfra, llama_3_2_1b_instruct_no_cache_deepinfra), # done 
    ("llama_3_2_3b_instruct", llama_3_2_3b_instruct_deepinfra, llama_3_2_3b_instruct_no_cache_deepinfra), # done
    ("llama_3_1_8b_instruct", llama_3_1_8b_instruct_deepinfra, llama_3_1_8b_instruct_no_cache_deepinfra),
    ("llama_3_1_70b_instruct", llama_3_1_70b_instruct_turbo_deepinfra, llama_3_1_70b_instruct_turbo_no_cache_deepinfra),

    ("qwen_2_5_7b_instruct", qwen_2_5_7b_instruct_deepinfra, qwen_2_5_7b_instruct_no_cache_deepinfra),
    ("qwen_2_5_72b_instruct", qwen_2_5_72b_instruct_deepinfra, qwen_2_5_72b_instruct_no_cache_deepinfra),
]


for llm_id, llm, llm_no_cache in to_do:
    program = dspy.Predict(TargetsExtractionSignature)
    
    save_and_evaluate(program=program, llm=llm, llm_no_cache=llm_no_cache, llm_id=llm_id, base_path=BASE_PATH, valset=devset)

In [9]:
# use for example vllm  https://github.com/vllm-project/vllm
API_KEY = None
BASE_URL = None
qwen_2_5_1p5b_instruct_vllm = dspy.LM("openai/Qwen/Qwen2.5-1.5B-Instruct",  api_key=API_KEY, base_url=BASE_URL, temperature=0.1, max_tokens=1024)
qwen_2_5_1p5b_instruct_no_cache_vllm = dspy.LM("openai/Qwen/Qwen2.5-1.5B-Instruct",  api_key=API_KEY, base_url=BASE_URL, temperature=0.3, max_tokens=1024, cache=False, cache_in_memory=False)


qwen_2_5_3b_instruct_vllm = dspy.LM(model="openai/Qwen/Qwen2.5-3B-Instruct", api_key=API_KEY, base_url=BASE_URL, temperature=0.1, max_tokens=1024)
qwen_2_5_3b_instruct_no_cache_vllm = dspy.LM(model="openai/Qwen/Qwen2.5-3B-Instruct", api_key=API_KEY, base_url=BASE_URL, temperature=0.3, max_tokens=1024, cache=False, cache_in_memory=False)

In [ ]:
assert False, "DONE"

to_do = [
    ("qwen_2_5_1p5b_instruct", qwen_2_5_1p5b_instruct_vllm, qwen_2_5_1p5b_instruct_no_cache_vllm)
]

BASE_PATH = "./TargetsExtractor/Baseline"

mlflow.autolog()
mlflow.set_experiment(BASE_PATH)

for llm_id, llm, llm_no_cache in to_do:
    program = dspy.Predict(TargetsExtractionSignature)
    
    save_and_evaluate(program=program, llm=llm, llm_no_cache=llm_no_cache, llm_id=llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"

to_do = [
    ("qwen_2_5_3b_instruct", qwen_2_5_3b_instruct_vllm, qwen_2_5_3b_instruct_no_cache_vllm)
]

BASE_PATH = "./TargetsExtractor/Baseline"

mlflow.autolog()
mlflow.set_experiment(BASE_PATH)

for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.Predict(TargetsExtractionSignature)
    save_and_evaluate(program=program, llm=llm, llm_no_cache=llm_no_cache, llm_id=llm_id, base_path=BASE_PATH, valset=devset)

# ZERO SHOT OPTIMIERUNG

In [ ]:
o1_optimizer_settings = dict(num_candidates = 10, max_bootstrapped_demos = 0, max_labeled_demos = 0, metric_threshold = None, init_temperature = 0.5, task_model = None, num_threads = 16, max_errors = 30, prompt_model=llama_3_1_70b_instruct_turbo_deepinfra, teacher_settings=dict(lm=llama_3_1_70b_instruct_turbo_deepinfra), auto=None)


o1_compiler_settings = dict(num_trials = 30, minibatch = True, minibatch_size = 25, minibatch_full_eval_steps = 10, program_aware_proposer=True, data_aware_proposer=True, view_data_batch_size=10, tip_aware_proposer=True, fewshot_aware_proposer=True, requires_permission_to_run=False)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"

BASE_PATH = "./TargetsExtractor/ZERO-O1"


to_do = [
    ("llama_3_2_1b_instruct", llama_3_2_1b_instruct_deepinfra, llama_3_2_1b_instruct_no_cache_deepinfra),
    ("llama_3_2_3b_instruct", llama_3_2_3b_instruct_deepinfra, llama_3_2_3b_instruct_no_cache_deepinfra),
    ("llama_3_1_8b_instruct", llama_3_1_8b_instruct_deepinfra, llama_3_1_8b_instruct_no_cache_deepinfra),
    ("llama_3_1_70b_instruct", llama_3_1_70b_instruct_turbo_deepinfra, llama_3_1_70b_instruct_turbo_no_cache_deepinfra),

    ("qwen_2_5_7b_instruct", qwen_2_5_7b_instruct_deepinfra, qwen_2_5_7b_instruct_no_cache_deepinfra),
    ("qwen_2_5_72b_instruct", qwen_2_5_72b_instruct_deepinfra, qwen_2_5_72b_instruct_no_cache_deepinfra), # tbd
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)

    o1_optimizer = MIPROv2(metric=metric, **o1_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o1_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o1_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"
BASE_PATH = "./TargetsExtractor/ZERO-O1"

to_do = [
    ("qwen_2_5_1p5b_instruct", qwen_2_5_1p5b_instruct_vllm, qwen_2_5_1p5b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)
    
    o1_optimizer = MIPROv2(metric=metric, **o1_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o1_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o1_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"
BASE_PATH = "./TargetsExtractor/ZERO-O1"


to_do = [
    ("qwen_2_5_3b_instruct", qwen_2_5_3b_instruct_vllm, qwen_2_5_3b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)
    
    o1_optimizer = MIPROv2(metric=metric, **o1_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o1_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o1_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

In [ ]:
o2_optimizer_settings = dict(num_candidates = 25, max_bootstrapped_demos = 0, max_labeled_demos = 0, metric_threshold = None, init_temperature = 0.5, task_model = None, num_threads = 24,max_errors = 30, prompt_model=llama_3_1_70b_instruct_turbo_deepinfra, teacher_settings=dict(lm=llama_3_1_70b_instruct_turbo_deepinfra), auto=None)


o2_compiler_settings = dict(num_trials = 50, minibatch = True, minibatch_size = 25, minibatch_full_eval_steps = 10, program_aware_proposer=True, data_aware_proposer=True, view_data_batch_size=10, tip_aware_proposer=True, fewshot_aware_proposer=True, requires_permission_to_run=False)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"

BASE_PATH = "./TargetsExtractor/ZERO-O2"


to_do = [
    ("llama_3_2_1b_instruct", llama_3_2_1b_instruct_deepinfra, llama_3_2_1b_instruct_no_cache_deepinfra),
    ("llama_3_2_3b_instruct", llama_3_2_3b_instruct_deepinfra, llama_3_2_3b_instruct_no_cache_deepinfra),
    ("llama_3_1_8b_instruct", llama_3_1_8b_instruct_deepinfra, llama_3_1_8b_instruct_no_cache_deepinfra),
    ("llama_3_1_70b_instruct", llama_3_1_70b_instruct_turbo_deepinfra, llama_3_1_70b_instruct_turbo_no_cache_deepinfra),

    ("qwen_2_5_7b_instruct", qwen_2_5_7b_instruct_deepinfra, qwen_2_5_7b_instruct_no_cache_deepinfra),
    ("qwen_2_5_72b_instruct", qwen_2_5_72b_instruct_deepinfra, qwen_2_5_72b_instruct_no_cache_deepinfra),
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)

    o2_optimizer = MIPROv2(metric=metric, **o2_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o2_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o2_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"
BASE_PATH = "./TargetsExtractor/ZERO-O2"


to_do = [
    ("qwen_2_5_1p5b_instruct", qwen_2_5_1p5b_instruct_vllm, qwen_2_5_1p5b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)
    
    o2_optimizer = MIPROv2(metric=metric, **o2_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o2_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o2_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"

BASE_PATH = "./TargetsExtractor/ZERO-O2"


to_do = [
    ("qwen_2_5_3b_instruct", qwen_2_5_3b_instruct_vllm, qwen_2_5_3b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)
    
    o2_optimizer = MIPROv2(metric=metric, **o2_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o2_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o2_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

# FEW SHOT OPTIMIERUNG O1

In [ ]:
o1_optimizer_settings = dict(num_candidates = 10, max_bootstrapped_demos = 1, max_labeled_demos = 2, metric_threshold = None, init_temperature = 0.5, task_model = None, num_threads = 64, max_errors = 100, prompt_model=llama_3_1_70b_instruct_turbo_deepinfra, teacher_settings=dict(lm=llama_3_1_70b_instruct_turbo_deepinfra), auto=None)


o1_compiler_settings = dict(num_trials = 30, minibatch = True, minibatch_size = 25, minibatch_full_eval_steps = 10, program_aware_proposer=True, data_aware_proposer=True, view_data_batch_size=10, tip_aware_proposer=True, fewshot_aware_proposer=True, requires_permission_to_run=False)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"

BASE_PATH = "./TargetsExtractor/FS-O1"

to_do = [
    ("llama_3_2_1b_instruct", llama_3_2_1b_instruct_deepinfra, llama_3_2_1b_instruct_no_cache_deepinfra),
    ("llama_3_2_3b_instruct", llama_3_2_3b_instruct_deepinfra, llama_3_2_3b_instruct_no_cache_deepinfra),
    ("llama_3_1_8b_instruct", llama_3_1_8b_instruct_deepinfra, llama_3_1_8b_instruct_no_cache_deepinfra),
    ("llama_3_1_70b_instruct", llama_3_1_70b_instruct_turbo_deepinfra, llama_3_1_70b_instruct_turbo_no_cache_deepinfra),

    ("qwen_2_5_7b_instruct", qwen_2_5_7b_instruct_deepinfra, qwen_2_5_7b_instruct_no_cache_deepinfra),
    ("qwen_2_5_72b_instruct", qwen_2_5_72b_instruct_deepinfra, qwen_2_5_72b_instruct_no_cache_deepinfra),
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)

    o1_optimizer = MIPROv2(metric=metric, **o1_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o1_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o1_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"
BASE_PATH = "./TargetsExtractor/FS-O1"


to_do = [
    ("qwen_2_5_1p5b_instruct", qwen_2_5_1p5b_instruct_vllm, qwen_2_5_1p5b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)
    
    o1_optimizer = MIPROv2(metric=metric, **o1_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o1_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o1_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"
BASE_PATH = "./TargetsExtractor/FS-O1"

to_do = [
    ("qwen_2_5_3b_instruct", qwen_2_5_3b_instruct_vllm, qwen_2_5_3b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)
    
    o1_optimizer = MIPROv2(metric=metric, **o1_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o1_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o1_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

# FEW SHOT OPTIMIERUNG O2

In [ ]:
o2_optimizer_settings = dict(num_candidates = 25, max_bootstrapped_demos = 1, max_labeled_demos = 2, metric_threshold = None, init_temperature = 0.5, task_model = None, num_threads = 24, max_errors = 100, prompt_model=llama_3_1_70b_instruct_turbo_deepinfra, teacher_settings=dict(lm=llama_3_1_70b_instruct_turbo_deepinfra), auto=None)


o2_compiler_settings = dict(num_trials = 50, minibatch = True, minibatch_size = 25, minibatch_full_eval_steps = 10, program_aware_proposer=True, data_aware_proposer=True, view_data_batch_size=10, tip_aware_proposer=True, fewshot_aware_proposer=True, requires_permission_to_run=False)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"

BASE_PATH = "./TargetsExtractor/FS-O2"

to_do = [
    ("llama_3_2_1b_instruct", llama_3_2_1b_instruct_deepinfra, llama_3_2_1b_instruct_no_cache_deepinfra),
    ("llama_3_2_3b_instruct", llama_3_2_3b_instruct_deepinfra, llama_3_2_3b_instruct_no_cache_deepinfra),
    ("llama_3_1_8b_instruct", llama_3_1_8b_instruct_deepinfra, llama_3_1_8b_instruct_no_cache_deepinfra),
    ("llama_3_1_70b_instruct", llama_3_1_70b_instruct_turbo_deepinfra, llama_3_1_70b_instruct_turbo_no_cache_deepinfra),

    ("qwen_2_5_7b_instruct", qwen_2_5_7b_instruct_deepinfra, qwen_2_5_7b_instruct_no_cache_deepinfra),
    ("qwen_2_5_72b_instruct", qwen_2_5_72b_instruct_deepinfra, qwen_2_5_72b_instruct_no_cache_deepinfra),
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)

    o2_optimizer = MIPROv2(metric=metric, **o2_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o2_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o2_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"

BASE_PATH = "./TargetsExtractor/FS-O2"

to_do = [
    ("qwen_2_5_1p5b_instruct", qwen_2_5_1p5b_instruct_vllm, qwen_2_5_1p5b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)
    
    o2_optimizer = MIPROv2(metric=metric, **o2_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o2_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o2_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

In [ ]:
assert False, "already finished and optimizer steps are not cached!"
BASE_PATH = "./TargetsExtractor/FS-O2"

to_do = [
    ("qwen_2_5_3b_instruct", qwen_2_5_3b_instruct_vllm, qwen_2_5_3b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    default_program = dspy.Predict(TargetsExtractionSignature)
    
    o2_optimizer = MIPROv2(metric=metric, **o2_optimizer_settings)
                    
    with dspy.settings.context(lm=llm):
        optimized_program = o2_optimizer.compile(student=default_program, trainset=trainset, valset=devset, **o2_compiler_settings)

    # store program (cloudpickle)
    optimized_program.save(f"{BASE_PATH}/{llm_id}", save_program=True)

# EVALUATION OF THE OPTIMIZED PROGRAMS

In [ ]:
assert False, "DONE"
BASE_PATH = "./TargetsExtractor/ZERO-O1"

to_do = [
    ("llama_3_2_1b_instruct", llama_3_2_1b_instruct_deepinfra, llama_3_2_1b_instruct_no_cache_deepinfra),
    ("llama_3_2_3b_instruct", llama_3_2_3b_instruct_deepinfra, llama_3_2_3b_instruct_no_cache_deepinfra),
    ("llama_3_1_8b_instruct", llama_3_1_8b_instruct_deepinfra, llama_3_1_8b_instruct_no_cache_deepinfra),
    ("llama_3_1_70b_instruct", llama_3_1_70b_instruct_turbo_deepinfra, llama_3_1_70b_instruct_turbo_no_cache_deepinfra),

    ("qwen_2_5_7b_instruct", qwen_2_5_7b_instruct_deepinfra, qwen_2_5_7b_instruct_no_cache_deepinfra),
    ("qwen_2_5_72b_instruct", qwen_2_5_72b_instruct_deepinfra, qwen_2_5_72b_instruct_no_cache_deepinfra),
]

for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")
    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"

BASE_PATH = "./TargetsExtractor/ZERO-O1"


to_do = [
    ("qwen_2_5_1p5b_instruct", qwen_2_5_1p5b_instruct_vllm, qwen_2_5_1p5b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")

    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"

BASE_PATH = "./TargetsExtractor/ZERO-O1"


to_do = [
    ("qwen_2_5_3b_instruct", qwen_2_5_3b_instruct_vllm, qwen_2_5_3b_instruct_no_cache_vllm)
]

for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")

    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"
BASE_PATH = "./TargetsExtractor/ZERO-O2"

to_do = [
    ("llama_3_2_1b_instruct", llama_3_2_1b_instruct_deepinfra, llama_3_2_1b_instruct_no_cache_deepinfra),
    ("llama_3_2_3b_instruct", llama_3_2_3b_instruct_deepinfra, llama_3_2_3b_instruct_no_cache_deepinfra),
    ("llama_3_1_8b_instruct", llama_3_1_8b_instruct_deepinfra, llama_3_1_8b_instruct_no_cache_deepinfra),
    ("llama_3_1_70b_instruct", llama_3_1_70b_instruct_turbo_deepinfra, llama_3_1_70b_instruct_turbo_no_cache_deepinfra),

    ("qwen_2_5_7b_instruct", qwen_2_5_7b_instruct_deepinfra, qwen_2_5_7b_instruct_no_cache_deepinfra),
    ("qwen_2_5_72b_instruct", qwen_2_5_72b_instruct_deepinfra, qwen_2_5_72b_instruct_no_cache_deepinfra),
]

for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")

    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"

BASE_PATH = "./TargetsExtractor/ZERO-O2"


to_do = [
    ("qwen_2_5_1p5b_instruct", qwen_2_5_1p5b_instruct_vllm, qwen_2_5_1p5b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")

    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"
BASE_PATH = "./TargetsExtractor/ZERO-O2"


to_do = [
    ("qwen_2_5_3b_instruct", qwen_2_5_3b_instruct_vllm, qwen_2_5_3b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")

    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

## FEW SHOT Evaluation

In [ ]:
assert False, "DONE"
BASE_PATH = "./TargetsExtractor/FS-O1"


to_do = [
    ("llama_3_2_1b_instruct", llama_3_2_1b_instruct_deepinfra, llama_3_2_1b_instruct_no_cache_deepinfra),
    ("llama_3_2_3b_instruct", llama_3_2_3b_instruct_deepinfra, llama_3_2_3b_instruct_no_cache_deepinfra),
    ("llama_3_1_8b_instruct", llama_3_1_8b_instruct_deepinfra, llama_3_1_8b_instruct_no_cache_deepinfra),
    ("llama_3_1_70b_instruct", llama_3_1_70b_instruct_turbo_deepinfra, llama_3_1_70b_instruct_turbo_no_cache_deepinfra),

    ("qwen_2_5_7b_instruct", qwen_2_5_7b_instruct_deepinfra, qwen_2_5_7b_instruct_no_cache_deepinfra),
    ("qwen_2_5_72b_instruct", qwen_2_5_72b_instruct_deepinfra, qwen_2_5_72b_instruct_no_cache_deepinfra),
]


for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")
    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"

BASE_PATH = "./TargetsExtractor/FS-O1"


to_do = [
    ("qwen_2_5_1p5b_instruct", qwen_2_5_1p5b_instruct_vllm, qwen_2_5_1p5b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")

    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"

BASE_PATH = "./TargetsExtractor/FS-O1"


to_do = [
    ("qwen_2_5_3b_instruct", qwen_2_5_3b_instruct_vllm, qwen_2_5_3b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")

    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"

BASE_PATH = "./TargetsExtractor/FS-O2"


to_do = [
    ("llama_3_2_1b_instruct", llama_3_2_1b_instruct_deepinfra, llama_3_2_1b_instruct_no_cache_deepinfra),
    ("llama_3_2_3b_instruct", llama_3_2_3b_instruct_deepinfra, llama_3_2_3b_instruct_no_cache_deepinfra),
    ("llama_3_1_8b_instruct", llama_3_1_8b_instruct_deepinfra, llama_3_1_8b_instruct_no_cache_deepinfra),
    ("llama_3_1_70b_instruct", llama_3_1_70b_instruct_turbo_deepinfra, llama_3_1_70b_instruct_turbo_no_cache_deepinfra),

    ("qwen_2_5_7b_instruct", qwen_2_5_7b_instruct_deepinfra, qwen_2_5_7b_instruct_no_cache_deepinfra),
    ("qwen_2_5_72b_instruct", qwen_2_5_72b_instruct_deepinfra, qwen_2_5_72b_instruct_no_cache_deepinfra),
]


for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")

    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"

BASE_PATH = "./TargetsExtractor/FS-O2"


to_do = [
    ("qwen_2_5_1p5b_instruct", qwen_2_5_1p5b_instruct_vllm, qwen_2_5_1p5b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")

    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)

In [ ]:
assert False, "DONE"

BASE_PATH = "./TargetsExtractor/FS-O2"


to_do = [
    ("qwen_2_5_3b_instruct", qwen_2_5_3b_instruct_vllm, qwen_2_5_3b_instruct_no_cache_vllm)
]


for llm_id, llm, llm_no_cache in to_do: 
    program = dspy.load(f"{BASE_PATH}/{llm_id}")
    save_and_evaluate(program, llm, llm_no_cache, llm_id, base_path=BASE_PATH, valset=devset)